In [ ]:
''' VMovies Title	Type	Overview	Cast	Director	Status	Orignal Language	Budget	Revenue	Star	Score
Black Widow	Action,Adventure,Thriller,Si Fi	Natasha Romanoff, also known as Black Widow,  .	Scarlett Johansson	Cate ShortLand	Released	English	$200M	$215M	8	100 '''

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# when erro 403 then this request is used
from urllib.request import Request, urlopen

url="https://www.themoviedb.org/movie"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

web_byte = urlopen(req).read()

webpage = web_byte.decode('utf-8')

In [8]:
soup = BeautifulSoup(webpage,"html.parser")
movie_details=soup.find_all('div',{'class':'card style_1'})

In [4]:
def pages(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    web_byte = urlopen(req).read()
    webpage = web_byte.decode('utf-8')
    soup = BeautifulSoup(webpage,"html.parser")
    return soup

In [5]:
page_list=[]
url='https://www.themoviedb.org/movie?page='
for i in range(6):
    page_list.append(url+str(i))
page_list

['https://www.themoviedb.org/movie?page=0',
 'https://www.themoviedb.org/movie?page=1',
 'https://www.themoviedb.org/movie?page=2',
 'https://www.themoviedb.org/movie?page=3',
 'https://www.themoviedb.org/movie?page=4',
 'https://www.themoviedb.org/movie?page=5']

In [6]:
movies_urls=[]
def get_movie_urls():
    for i in range(len(page_list)):
        soup = pages(page_list[i])
        movie_details=soup.find_all('div',{'class':'card style_1'})
        baseUrl= 'https://www.themoviedb.org'
        for link in movie_details:
            movies_urls.append(baseUrl + link.a['href'])
    return movies_urls

len(get_movie_urls())

100

In [9]:
movies_link=[]
baseUrl= 'https://www.themoviedb.org'
for link in movie_details:
    movies_link.append(baseUrl + link.a['href'])
movies_link[4]

'https://www.themoviedb.org/movie/385128'

In [10]:
movie_date=[]
for date in movie_details:
    movie_date.append(date.p.text)
movie_date[1]

'Jul 07, 2021'

In [12]:
def get_movie_webpage():
    movie_url = 'https://www.themoviedb.org/movie'
    req = Request(movie_url, headers={'User-Agent': 'Mozilla/5.0'})
    web_byte = urlopen(req).read()
    webpage = web_byte.decode('utf-8')
    
    soup=BeautifulSoup(webpage,'html.parser')
    return soup 

In [15]:
def get_movies_titles(soup):
    movie_details=soup.find_all('div',{'class':'card style_1'})
    movie_title = []
    for title in movie_details:
        movie_title.append(title.a['title']) 
    return movie_title

In [18]:
# get_movies_titles(soup)

In [23]:
def get_movies_urls():
    movies_link=[]
    baseUrl= 'https://www.themoviedb.org'
    for link in movie_details:
        movies_link.append(baseUrl + link.a['href'])
    return movies_link

In [22]:
len(get_movie_urls())

200

In [24]:
def get_movies_detail():
    movies_dict = {
        'title': get_movies_titles(soup),
        'url': get_movies_urls(soup)
    }
    return pd.DataFrame(movies_dict)

In [25]:
url= 'https://www.themoviedb.org/movie/497698-black-widow'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

web_byte = urlopen(req).read()

webpage = web_byte.decode('utf-8')
soup=BeautifulSoup(webpage,'html.parser')
movie_date = soup.find('div',{'class':'facts'})
x=movie_date.find_all('span')
date=x[1].text

In [26]:
date.strip()

'07/09/2021 (US)'

In [38]:
def get_movie_page(movie_url):
    # Download the page
    
    req = Request(movie_url, headers={'User-Agent': 'Mozilla/5.0'})
    web_byte = urlopen(req).read()
    webpage = web_byte.decode('utf-8')
    movie_soup = BeautifulSoup(webpage, 'html.parser')
    return movie_soup

In [29]:
len(get_movie_page(movies_link[0]))

3

In [39]:
def get_movie_detail(movie_url):
    soup=get_movie_page(movie_url)
    movie_title=soup.find('div',{'class':'title ott_false'})
    title=movie_title.a.text
    
    movie_date = soup.find('div',{'class':'facts'})
    x=movie_date.find_all('span')
    date=x[1].text.strip()
    
    movie_Overviews=soup.find('div',{'class':'overview'})
    movie_Overview = movie_Overviews.text.strip()
    
    movie_directors = soup.find('ol',{'class':'people no_image'})
    movie_director=movie_directors.p.text
    
    movie_cast = soup.find('ol',{'class':'people scroller'})
    cast=movie_cast.find_all('li')
    
    cast_derail = [c.text.strip() for c in cast[:9]]
   
    status_all=soup.find('div',{'class':'grey_column'})
    p_tag=status_all.find_all('p')
    for d in p_tag:
        status=p_tag[0].text.strip()
        orignal_language=p_tag[1].text.strip()
        budget = p_tag[2].text.strip()
        revenue=p_tag[3].text.strip()
        score= p_tag[4].text.strip()
        break
        
    return title,date,movie_director,movie_Overview,cast_derail,status,orignal_language,budget,revenue,score

In [40]:
# get_movie_detail(movies_link[0])

In [41]:
def get_allMovie():
    movie_dict ={ 'Title':[],
                 'Date': [],
            'Director': [],
            'Overview': [],
            'Cast': [],
            'Status': [],
            'Orignal Language': [],
            'Budget': [],
            'Revenue': [],
            'Score': [],
           }

    # Get repo info
    for i in range(20):
        movie_info = get_movie_detail(movies_link[i])
        movie_dict['Title'].append(movie_info[0])
        movie_dict['Date'].append(movie_info[1])
        movie_dict['Director'].append(movie_info[2])
        movie_dict['Overview'].append(movie_info[3])
        movie_dict['Cast'].append(movie_info[4])
        movie_dict['Status'].append(movie_info[5])
        movie_dict['Orignal Language'].append(movie_info[6])
        movie_dict['Budget'].append(movie_info[7])
        movie_dict['Revenue'].append(movie_info[8])
        movie_dict['Score'].append(movie_info[9])
        
    return pd.DataFrame(movie_dict)

In [42]:
get_allMovie()

,Title,Date,Director,Overview,Cast,Status,Orignal Language,Budget,Revenue,Score
0,Space Jam: A New Legacy,07/16/2021 (US),Malcolm D. Lee,When LeBron and his young son Dom are trapped ...,"[LeBron James\nLeBron James, Don Cheadle\nAl G...",Status Released,Original Language English,"Budget $161,900,000.00","Revenue $93,967,605.00",100
1,Black Widow,07/09/2021 (US),Cate Shortland,"Natasha Romanoff, also known as Black Widow, c...",[Scarlett Johansson\nNatasha Romanoff / Black ...,Status Released,Original Language English,"Budget $200,000,000.00","Revenue $318,073,697.00",100
2,The Forever Purge,07/02/2021 (US),Everardo Gout,All the rules are broken as a sect of lawless ...,"[Josh Lucas\nDylan Tucker, Ana de la Reguera\n...",Status Released,Original Language English,"Budget $18,000,000.00","Revenue $52,802,945.00",100
3,The Boss Baby: Family Business,07/02/2021 (US),Michael McCullers,The Templeton brothers — Tim and his Boss Baby...,"[Alec Baldwin\nTheodore Templeton (voice), Jam...",Status Released,Original Language English,"Budget $82,000,000.00","Revenue $51,128,635.00",100
4,F9,06/25/2021 (US),Justin Lin,Dominic Toretto and his crew battle the most s...,"[Vin Diesel\nDominic Toretto, Michelle Rodrigu...",Status Released,Original Language English,"Budget $200,000,000.00","Revenue $624,455,615.00",100
5,Luca,06/17/2021 (RU),Enrico Casarosa,Luca and his best friend Alberto experience an...,"[Jacob Tremblay\nLuca Paguro (voice), Jack Dyl...",Status Released,Original Language English,Budget -,"Revenue $11,600,000.00",100
6,The Tomorrow War,07/01/2021 (US),Chris McKay,The world is stunned when a group of time trav...,"[Chris Pratt\nDan Forester, Yvonne Strahovski\...",Status Released,Original Language English,Budget -,Revenue -,100
7,Dynasty Warriors,04/29/2021 (TW),Roy Chow Hin-Yeung,"Warlords, warriors and statesmen wage a battle...","[Louis Koo\nLu Bu, Nick Wang\nCao Cao, Ray Lui...",Original Title 真·三国无双,Status Released,Original Language Cantonese,Budget -,Revenue -
8,Infinite,06/10/2021 (FR),Antoine Fuqua,Evan McCauley has skills he never learned and ...,"[Mark Wahlberg\nEvan McCauley, Chiwetel Ejiofo...",Status Released,Original Language English,Budget -,Revenue -,100
9,The Exorcism of Carmen Farias,Horror,Rodrigo Fiallega,"Carmen, a brave journalist, discovers soon aft...","[Camila Sodi\nCarmen Farías, Lucy Paez\nCarmen...",Original Title El exorcismo de Carmen Farías,Status Released,Original Language Spanish; Castilian,Budget -,"Revenue $1,474,579.00"


In [43]:
def scrape_Moviess():
    if os.path.exists(path):
        print("The file {} already exists. Skipping...".format(path))
        return
    topic_df = get_allMovie()
    topic_df.to_csv('moviesDetail.csv', index=None)

In [163]:
scrape_Moviess()